In [ ]:
!pip install ta

: 

In [1]:
import sys
import os

# Add the root directory "chaoticX" to the system path
sys.path.append(os.path.abspath(".."))  # one level up from Notebooks/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
from Data.timeFrames import timeFrame
from Data.binanceAPI import BinanceAPI
from Core.zone_detection import ZoneDetector
from Core.zone_merge import ZoneMerger
from Core.zone_reactions import ZoneReactor


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/OHLC.csv')

In [3]:
api = BinanceAPI()
df_4h = api.get_ohlcv(interval='4h')
df_1d = api.get_ohlcv(interval='1D')
df_1h = api.get_ohlcv()

In [4]:
detector1h = ZoneDetector(df_1h)
zone_1h = detector1h.get_zones()
detector4h = ZoneDetector(df_4h,timeframe= "4h")
zone_4h = detector4h.get_zones()
detector1d = ZoneDetector(df_1d,timeframe="1D")
zone_1d = detector1d.get_zones()


In [5]:
merger = ZoneMerger(zone_1h+zone_4h+zone_1d)

zones = merger.merge()
#zones = merger.add_liq_confluence(zones)

In [6]:
reactor = ZoneReactor(df_1h, zones)
zones = reactor.get_zone_reaction()
zones = merger.getNearbyZone(zones)

In [8]:
zones = reactor.get_next_target_zone()

UnboundLocalError: cannot access local variable 'zone_copy' where it is not associated with a value

In [7]:
zones[-1]

{'zone_high': np.float32(113067.31),
 'zone_low': np.float32(102969.484),
 'zone_width': np.float32(10097.828),
 'types': ['Bearish OB',
  'Sell-Side Liq',
  'Buy-Side Liq',
  'Bullish OB',
  'Bearish FVG'],
 'timeframes': ['4h', '1h', '1D'],
 'count': 10,
 'start_index': 21168,
 'end_index': 25848,
 'mid_index': 23574,
 'touch_type': 'body_close_above',
 'touch_candle': open        113722.828125
 high        113760.968750
 low         112985.640625
 close       113420.000000
 volume        1310.784546
 ema20       111490.173210
 ema50       110466.427775
 atr            562.486114
 rsi             82.061862
 atr_mean       422.322237
 Name: 2025-07-10 18:00:00, dtype: float64,
 'touch_index': 26094,
 'distance_to_nearest_zone_above': inf,
 'nearest_zone_above': None,
 'distance_to_nearest_zone_below': np.float32(439.96094),
 'nearest_zone_below': {'zone_high': np.float32(102529.52),
  'zone_low': np.float32(96911.47),
  'zone_width': np.float32(5618.0547),
  'types': ['Bearish FVG', '

In [ ]:
import requests

def get_orderbook_price(symbol, target_price):
    url = "https://api.binance.com/api/v3/depth"
    params = {"symbol": symbol.upper(), "limit": 10000}
    res = requests.get(url, params=params).json()

    target_price = str(float(target_price))  # Ensure formatting

    bids = {price: qty for price, qty in res["bids"]}
    asks = {price: qty for price, qty in res["asks"]}

    bid_qty = bids.get(target_price, None)
    ask_qty = asks.get(target_price, None)

    return {
        "bid_quantity": bid_qty,
        "ask_quantity": ask_qty,
        "response" : res
    }

# Example usage:
print(get_orderbook_price("BTCUSDT", 109000.00))


In [ ]:
df.to_csv('OHLC.csv')

In [ ]:
df_4h.tail(5)

In [ ]:
zone_1d

In [ ]:
merger.liq_zones

In [ ]:
zone_df = pd.DataFrame(merger.core_zones)
zone_df.to_csv('core_zone.csv')

In [ ]:
def check_fvg_reactions(df, fvg_list, lookahead=24, N=24):
    results = []

    for fvg in fvg_list:
        idx = fvg['index']
        high = fvg['zone_high']
        low = fvg['zone_low']
        direction = fvg['type']
        entered = False
        bounced = False
        violated = False
        partial_mitigated = False
        time_to_entry = None
        price_retrace_ratio = None
        volume_on_entry = None
        bounce_candles = None
        violated_candles = None
        max_price_move = None  # NEW
        reaction_type = 'no entry'

        for j in range(idx + 2, min(idx + 2 + lookahead, len(df))):
            candle = df.iloc[j]

            wick_enters = candle['low'] <= high and candle['high'] >= low
            close_inside = low <= candle['close'] <= high

            if wick_enters:
                entered = True
                entry_price = candle['low'] if direction == 'Bullish FVG' else candle['high']
                price_retrace_ratio = (entry_price - low) / (high - low) if direction == 'Bullish FVG' else (high - entry_price) / (high - low)
                volume_on_entry = candle['volume']
                if time_to_entry is None:
                    time_to_entry = j - idx

                if not close_inside:
                    partial_mitigated = True
                    reaction_type = 'partial mitigate'

                if direction == 'Bullish FVG':
                    if candle['close'] > high:
                        bounced = True
                        bounce_candles = j - idx - time_to_entry


                        for k in range(j+1,min(j +N, len(df))):
                            # Check if price reverses (violation)
                            if df.iloc[k]['close'] < low:
                                violated = True
                                violated_candles = k-j
                                # Look ahead for max price movement after bounce
                                max_high = df.iloc[j : k]['high'].max()
                                max_price_move = max_high - high
                                reaction_type = 'bounce + break'
                                break
                        if not violated:
                            # Look ahead for max price movement after bounce
                            max_high = df.iloc[j : j + N]['high'].max()
                            max_price_move = max_high - high
                            reaction_type = 'clean bounce'
                        break

                    elif candle['close'] < low:
                        violated = True
                        violated_candles = j-time_to_entry-idx
                        reaction_type = 'violation'
                        break

                elif direction == 'Bearish FVG':
                    if candle['close'] < low:
                        bounced = True
                        bounce_candles = j - idx -time_to_entry
                        for k in range(j+1,min(j +N, len(df))):
                            # Check if price reverses (violation)
                            if df.iloc[k]['close'] > high:
                                violated = True
                                violated_candles = k-j
                                min_low = df.iloc[j : k]['low'].min()
                                max_price_move = low - min_low
                                reaction_type = 'bounce + break'
                                break
                        if not violated:
                            min_low = df.iloc[j : j+N]['low'].min()
                            max_price_move = low - min_low
                            reaction_type = 'clean bounce'
                        break

                    elif candle['close'] > high:
                        violated = True
                        violated_candles = j-time_to_entry-idx
                        reaction_type = 'violation'
                        break


        results.append({
            **fvg,
            'entered': entered,
            'partial_mitigated': partial_mitigated and not close_inside,
            'bounced': bounced,
            'violated': violated,
            'time_to_entry' : time_to_entry,
            'price_retrace_ratio' : price_retrace_ratio,
            'volume_on_entry' : volume_on_entry,
            'bounce_after_candles': bounce_candles,
            'violated_after_candles' : violated_candles,
            'max_price_move': max_price_move,  # NEW
            'reaction_type' : reaction_type
        })

    return results


In [ ]:
def check_ob_reactions(df, ob_list, lookahead=24, N=24):
    results = []

    for ob in ob_list:
        idx = ob['index']
        high = ob['zone_high']
        low = ob['zone_low']
        direction = ob['type']
        entered = False
        bounced = False
        violated = False
        partial_mitigated = False
        time_to_entry = None
        price_retrace_ratio = None
        volume_on_entry = None
        bounce_candles = None
        violated_candles = None
        max_price_move = None  # NEW
        reaction_type = 'no entry'

        for j in range(idx + 2, min(idx + 2 + lookahead, len(df))):
            candle = df.iloc[j]

            wick_enters = candle['low'] <= high and candle['high'] >= low
            close_inside = low <= candle['close'] <= high

            if wick_enters:
                entered = True
                entry_price = candle['low'] if direction == 'Bullish OB' else candle['high']
                price_retrace_ratio = (entry_price - low) / (high - low) if direction == 'Bullish OB' else (high - entry_price) / (high - low)
                volume_on_entry = candle['volume']
                if time_to_entry is None:
                    time_to_entry = j - idx

                if not close_inside:
                    partial_mitigated = True
                    reaction_type = 'partial mitigate'

                if direction == 'Bullish OB':
                    if candle['close'] > high:
                        bounced = True
                        bounce_candles = j - time_to_entry -idx


                        for k in range(j+1,min(j +N, len(df))):
                            # Check if price reverses (violation)
                            if df.iloc[k]['close'] < low:
                                violated = True
                                violated_candles = k-j
                                # Look ahead for max price movement after bounce
                                max_high = df.iloc[j : k]['high'].max()
                                max_price_move = max_high - high
                                reaction_type = 'bounce + break'
                                break
                        if not violated:
                            # Look ahead for max price movement after bounce
                            max_high = df.iloc[j : j + N]['high'].max()
                            max_price_move = max_high - high
                            reaction_type = 'clean bounce'
                        break

                    elif candle['close'] < low:
                        violated = True
                        violated_candles = j-time_to_entry-idx
                        reaction_type = 'violation'
                        break

                elif direction == 'Bearish OB':
                    if candle['close'] < low:
                        bounced = True
                        bounce_candles = j - time_to_entry - idx
                        for k in range(j+1,min(j +N, len(df))):
                            # Check if price reverses (violation)
                            if df.iloc[k]['close'] > high:
                                violated = True
                                violated_candles = k-j
                                min_low = df.iloc[j : k]['low'].min()
                                max_price_move = low - min_low
                                reaction_type = 'bounce + break'
                                break
                        if not violated:
                            min_low = df.iloc[j : j+N]['low'].min()
                            max_price_move = low - min_low
                            reaction_type = 'clean bounce'
                        break

                    elif candle['close'] > high:
                        violated = True
                        violated_candles = j-time_to_entry-idx
                        reaction_type = 'violation'
                        break


        results.append({
            **ob,
            'entered': entered,
            'partial_mitigated': partial_mitigated and not close_inside,
            'bounced': bounced,
            'violated': violated,
            'time_to_entry' : time_to_entry,
            'price_retrace_ratio' : price_retrace_ratio,
            'volume_on_entry' : volume_on_entry,
            'bounce_after_candles': bounce_candles,
            'violated_after_candle' : violated_candles,
            'max_price_move': max_price_move,  # NEW
            'reaction_type' : reaction_type
        })

    return results

In [ ]:
def check_liquidity_reactions_on_touches(df, liquidity_zones, lookahead=24, max_touches=5):
    results = []

    for zone in liquidity_zones:
        zone_high = zone['zone_high']
        zone_low = zone['zone_low']
        ltype = zone['type']

        touches = []
        touch_count = 0
        start = zone['start_index']

        for i in range(zone['start_index'] + 1, len(df)):
            low = df['low'].iloc[i]
            high = df['high'].iloc[i]
            volume = df['volume'].iloc[i]
            open_price = df['open'].iloc[i]

            if (high >= zone_low and open_price < zone_low) or ( open_price > zone_high and low <= zone_high):
                '''if touch_count >= max_touches:
                    break'''
                touch_index = i
                touch_result = {
                    'touch_index': touch_index,
                    'touch_volume' : volume,
                    'time_to_entry' : start - i,
                    'reaction_type': 'neutral',
                    'bounced': False,
                    'violated': False,
                    'price_move_after_touch': None
                }

                if touch_index + lookahead < len(df):
                    future = df.iloc[touch_index + 1 : touch_index + 1 + lookahead]

                    if ltype == 'Buy-Side Liq':
                        if any(future['close'] < zone_low):
                            touch_result['reaction_type'] = 'violation'
                            touch_result['violated'] = True
                        elif any(future['close'] > df['close'].iloc[touch_index]):
                            touch_result['reaction_type'] = 'bounce'
                            touch_result['bounced'] = True
                        else:
                            touch_result['reaction_type'] = 'neutral'
                        touch_result['price_move_after_touch'] = future['high'].max() - df['high'].iloc[touch_index]

                    elif ltype == 'Sell-Side Liq':
                        if any(future['close'] > zone_high):
                            touch_result['reaction_type'] = 'violation'
                            touch_result['violated'] = True
                        elif any(future['close'] < df['close'].iloc[touch_index]):
                            touch_result['reaction_type'] = 'bounce'
                            touch_result['bounced'] = True
                        else:
                            touch_result['reaction_type'] = 'neutral'
                        touch_result['price_move_after_touch'] = df['low'].iloc[touch_index] - future['low'].min()

                touches.append(touch_result)
                touch_count += 1
                start = i

        results.append({
            **zone,
            'touch_count': len(touches),
            'touches': touches
        })

    return results


In [ ]:
def add_distance_to_nearest_zones_above_below(zones, threshold=0.1):
    """
    Adds the nearest zone above and below the current zone based on mid-price distance.
    Each zone must have: 'zone_high', 'zone_low', 'type', and either 'index' or 'start_index'.
    """
    updated_zones = []

    for i, zone in enumerate(zones):
        this_mid = (zone['zone_high'] + zone['zone_low']) / 2

        if zone['type'] in ['Sell-Side Liq','Buy-Side Liq']:
          for index in zone['touch_indexs']:

            min_dist_above = float('inf')
            min_dist_below = float('inf')

            nearest_above_type = None
            nearest_below_type = None
            nearest_above_zone = None
            nearest_below_zone = None
            nearest_above_index = None
            nearest_below_index = None
            h1=None
            h2=None
            l1=None
            l2=None

            valid_zones = [
        z for z in zones
        if z.get('touch_index') is None or z.get('touch_index') > index or (z.get('swept_index',0) > index)
    ]


            for j, other in enumerate(valid_zones):
                if i == j:
                    continue

                other_mid = (other['zone_high'] + other['zone_low']) / 2
                dist = abs(this_mid - other_mid)
                if dist > (this_mid * threshold):
                    continue

                if other_mid > this_mid and dist < min_dist_above:
                    min_dist_above = dist
                    nearest_above_zone = other

                elif other_mid < this_mid and dist < min_dist_below:
                    min_dist_below = dist
                    nearest_below_zone = other

            updated = zone.copy()
            updated['distance_to_nearest_zone_above'] = min_dist_above
            updated['nearest_zone_above'] = nearest_above_zone

            updated['distance_to_nearest_zone_below'] = min_dist_below
            updated['nearest_zone_below'] = nearest_below_zone
            updated['liq_touch_idx'] = index

            #updated['valid_zones'] = valid_zones

            updated_zones.append(updated)

        else:

          min_dist_above = float('inf')
          min_dist_below = float('inf')

          nearest_above_type = None
          nearest_below_type = None
          nearest_above_zone = None
          nearest_below_zone = None
          nearest_above_index = None
          nearest_below_index = None
          h1=None
          h2=None
          l1=None
          l2=None

          valid_zones = [
      z for z in zones
      if z.get('touch_index') is None or z.get('touch_index') > zone.get('index', 0) or (z.get('swept_index',0) > zone.get('index',0))
  ]


          for j, other in enumerate(valid_zones):
              if i == j:
                  continue

              other_mid = (other['zone_high'] + other['zone_low']) / 2
              dist = abs(this_mid - other_mid)
              if dist > (this_mid * threshold):
                  continue

              if other_mid > this_mid and dist < min_dist_above:
                  min_dist_above = dist
                  nearest_above_zone = other

              elif other_mid < this_mid and dist < min_dist_below:
                  min_dist_below = dist
                  nearest_below_zone = other

          updated = zone.copy()
          updated['distance_to_nearest_zone_above'] = min_dist_above
          updated['nearest_zone_above'] = nearest_above_zone

          updated['distance_to_nearest_zone_below'] = min_dist_below
          updated['nearest_zone_below'] = nearest_below_zone

          #updated['valid_zones'] = valid_zones

          updated_zones.append(updated)

    return updated_zones


In [ ]:
all_zones = fvg+ob+liq
all_zones = add_distance_to_nearest_zones_above_below(all_zones)

In [ ]:
all_zones = sorted(all_zones,key = lambda x : x['index'])

In [ ]:
all_zones[-1]

In [ ]:
all_zones_reactions = check_fvg_reactions(df,fvg) + check_ob_reactions(df,ob) + check_liquidity_reactions_on_touches(df,liq)

In [ ]:
all_zones_reactions = add_distance_to_nearest_zones_above_below(all_zones_reactions)

In [ ]:
df_zone_reaction = pd.DataFrame(all_zones_reactions)
df_zone_reaction.head()

In [ ]:
df_zone_reaction.columns

In [ ]:
df_zone_reaction = df_zone_reaction.drop(columns=['entered',
'partial_mitigated',
'bounced',
'violated',
'price_retrace_ratio',
'bounce_after_candles',
'violated_after_candles','violated_after_candle','liquidity_height','avg_swing_strength',
'max_price_move'])

In [ ]:
df_zone_reaction.loc[df_zone_reaction['start_index'].notna(),'index'] = df_zone_reaction.loc[df_zone_reaction['start_index'].notna(),'index'].fillna(df_zone_reaction['start_index'])

In [ ]:
df_zone_reaction = df_zone_reaction.drop(columns=['start_index','end_index','swept_index','count','touch_count'])

In [ ]:
def flatten_zone_reaction_row(row):
    """
    Flattens a single liquidity zone row into multiple rows based on touches.

    Args:
        row (pd.Series): A row from a DataFrame containing a 'touches' column with list of dicts.

    Returns:
        List of dicts (flattened rows).
    """
    flattened = []

    touches = row.get('touches', [])
    if not isinstance(touches, list) or len(touches) == 0:
        # Return the row as-is with no touch-specific data
        base = row.drop(labels='touches').to_dict()
        flattened.append(base)
        return flattened

    for touch in touches:
        base = row.drop(labels='touches').to_dict()
        base['reaction_type'] = touch.get('reaction_type')
        base['time_to_entry'] = touch.get('time_to_entry')
        base['volume_on_entry'] = touch.get('touch_volume')
        flattened.append(base)

    return flattened

def flatten_reaction_dataframe(df):
    """
    Apply flatten_liquidity_row to each row and combine all into a new flattened DataFrame.
    """
    all_rows = []
    for _, row in df.iterrows():
        flattened_rows = flatten_zone_reaction_row(row)
        all_rows.extend(flattened_rows)
    return pd.DataFrame(all_rows)


In [ ]:
df_zone_reaction = flatten_reaction_dataframe(df_zone_reaction)

In [ ]:
df_zone_reaction['level'] = df_zone_reaction['level'].fillna((df_zone_reaction['zone_high']+df_zone_reaction['zone_low'])/2)
df_zone_reaction['zone_width'] = df_zone_reaction['zone_width'].fillna(0)
df_zone_reaction['body_size'] = df_zone_reaction['body_size'].fillna(0)
df_zone_reaction['wick_ratio'] = df_zone_reaction['wick_ratio'].fillna(0)
df_zone_reaction['volume_on_creation'] = df_zone_reaction['volume_on_creation'].fillna(0)
df_zone_reaction['avg_volume_past_5'] = df_zone_reaction['avg_volume_past_5'].fillna(0)
df_zone_reaction['prev_volatility_5'] = df_zone_reaction['prev_volatility_5'].fillna(0)
df_zone_reaction['momentum_5'] = df_zone_reaction['momentum_5'].fillna(0)
df_zone_reaction['avg_volume_around_zone'] = df_zone_reaction['avg_volume_around_zone'].fillna(0)
df_zone_reaction['equal_level_deviation'] = df_zone_reaction['equal_level_deviation'].fillna(0)
df_zone_reaction['duration_between_first_last_touch'] = df_zone_reaction['duration_between_first_last_touch'].fillna(0)

In [ ]:
df_zone_reaction['type'].unique()

In [ ]:
df_zone_reaction.to_csv('reactions.csv',index=False)

In [ ]:
df_zone_reaction.describe()

In [ ]:
df_zone_reaction.notna().sum()

In [ ]:
df_zone_reaction= df_zone_reaction.dropna()

In [ ]:
df_ob_fvg_reaction = df_zone_reaction.loc[~df_zone_reaction['type'].isin(['Buy-Side Liq','Sell-Side Liq'])]
df_liq_reaction = df_zone_reaction.loc[df_zone_reaction['type'] .isin(['Buy-Sidde Liq','Sell-Side Liq'])]

In [ ]:
reaction_type1 = list(df_ob_fvg_reaction.reaction_type.unique())
reaction_type2 = list(df_liq_reaction.reaction_type.unique())
zone_type = list(df_zone_reaction['type'].unique())

In [ ]:
df_ob_fvg_reaction['type'] = df_ob_fvg_reaction['type'].apply(lambda x : zone_type.index(x))
df_ob_fvg_reaction['nearest_zone_type'] = df_ob_fvg_reaction['nearest_zone_type'].apply(lambda x : zone_type.index(x))
df_liq_reaction['type'] = df_liq_reaction['type'].apply(lambda x : zone_type.index(x))
df_liq_reaction['nearest_zone_type'] = df_liq_reaction['nearest_zone_type'].apply(lambda x : zone_type.index(x))



In [ ]:
df_ob_fvg_reaction['reaction_type'] = df_ob_fvg_reaction['reaction_type'].apply(lambda x : reaction_type1.index(x))
df_liq_reaction['reaction_type'] = df_liq_reaction['reaction_type'].apply(lambda x : reaction_type2.index(x))

In [ ]:
df_zone_reaction.dtypes

In [ ]:
X1 = df_liq_reaction.drop(columns=['reaction_type','nearest_zone_index','index','zone_high','zone_low','duration_between_first_last_touch'])
Y1 = df_liq_reaction['reaction_type']
X2 = df_ob_fvg_reaction.drop(columns=['reaction_type','nearest_zone_index','index','zone_high','zone_low','duration_between_first_last_touch'])
Y2 = df_ob_fvg_reaction['reaction_type']

In [ ]:
from sklearn.model_selection import train_test_split

train_x1,val_x1,train_y1,val_y1 = train_test_split(X1,Y1,test_size=0.7)
train_x2,val_x2,train_y2,val_y2 = train_test_split(X2,Y2,test_size=0.7)

In [ ]:
from xgboost import XGBClassifier

liq_model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
ob_fvg_model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
liq_model.fit(train_x1, train_y1)
ob_fvg_model.fit(train_x2,train_y2)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# After training
y_pred1 = liq_model.predict(val_x1)
y_pred2 = ob_fvg_model.predict(val_x2)

print(classification_report(val_y1, y_pred1))
print(classification_report(val_y2, y_pred2))


In [ ]:
print(X2.columns)
print(X1.columns)

In [ ]:
def label_direction_based_on_zone_touch(df, fvg, ob, liq,  threshold=0.25):
    labels = []
    df = df.reset_index(drop=True)
    n = len(df)

    all_zones = []

    for zone_list, zone_type in [(fvg, 'FVG'), (ob, 'OB'), (liq, 'LIQ')]:
        for z in zone_list:
            if 'zone_high' not in z:
                z['zone_high'] = z.get('zone_high', z.get('Price', 0) + 0.001)
            if 'zone_low' not in z:
                z['zone_low'] = z.get('zone_low', z.get('Price', 0) - 0.001)
            z['zone_type'] = zone_type if not (zone_type== 'Swing') else z['Type']
            z['mid_price'] = (z['zone_high'] + z['zone_low']) / 2
            z['index'] = z.get('index', z.get('start_index'))
            if z['index'] is not None:
                all_zones.append(z)

    used_indx = set()
    i = 0
    while i < n:
        print(f"Processing candle {i+1}/{n}", end='\r')
        curr_price = df.loc[i, 'close']

        # Filter past zones once per i
        past_zones = [
            z for z in all_zones
            if z['index'] < i and (z['zone_high'], z['zone_low'], z['zone_type']) not in used_indx
            and abs(z['mid_price'] - curr_price) < curr_price*threshold
            or (z['zone_type'] == 'LIQ' and not z['swept_index']==None and z['swept_index']>i and z['index'] < i and abs(z['mid_price'] - curr_price) < curr_price*threshold)
        ]

        if not past_zones:
            labels.append({
                'index': i,
                'candle detail': df.iloc[i].to_dict(),
                'direction': None,
                'touched_index' : None,
                'touched_zone_type': None,
                'target_zone_index': None,
                'zone_detail' : None,
                'past_zones' : None
            })
            i += 1
            continue

        touched_zone = None
        touched_index = None

        for zone in past_zones:
            z_high = zone['zone_high']
            z_low = zone['zone_low']

            # Use slicing and vectorized comparison instead of for-loop
            future_slice = df.iloc[i+1:]
            condition = ((future_slice['low'] <= z_high) & (future_slice['open'] > z_high)) | ((future_slice['open'] < z_low) & (future_slice['high'] >= z_low))
            match = future_slice[condition]

            if not match.empty:
                touched_index = match.index[0]
                touched_zone = zone
                if(touched_zone['zone_type'] in ['FVG','Swing High','Swing Low','OB']):
                    used_indx.add((zone['zone_high'], zone['zone_low'], zone['zone_type']))


        if touched_zone and touched_index:
            price_slice = df.loc[i:touched_index - 1].copy()
            zone_mid = touched_zone['mid_price']
            direction_series = np.where(zone_mid > price_slice['close'], 'long', 'short')

            # Build labels all at once
            for j, (idx, row) in enumerate(price_slice.iterrows()):
                labels.append({
                    'index': idx,
                    'candle detail': row.to_dict(),
                    'direction': direction_series[j],
                    'touched_index' : touched_index,
                    'touched_zone_type': touched_zone['zone_type'],
                    'target_zone_index': touched_zone['index'],
                    'zone_detail' : touched_zone,
                    'past_zones' : past_zones
                })

            # Advance i
            i = touched_index + 1  # skip to after the touch
        else:
            labels.append({
                'index': i,
                'candle detail': df.iloc[i].to_dict(),
                'direction': 'none',
                'touched_index' : None,
                'touched_zone_type': None,
                'target_zone_index': None,
                'zone_detail' : None,
                'past_zones' : past_zones
            })
            i += 1

    return labels


In [ ]:
labels = label_direction_based_on_zone_touch(df,fvg,ob,liq,threshold=0.1)

In [ ]:
# Assuming 'labels' is the result from your function
df_labels = pd.DataFrame(labels)
# Expand 'candle detail'
candle_df = pd.json_normalize(df_labels['candle detail'])

# Expand 'touched_zone_detail'
zone_df = pd.json_normalize(df_labels['zone_detail'])

# Combine with top-level fields
df_final = pd.concat([df_labels.drop(columns=['candle detail', 'zone_detail']),
                      candle_df.add_prefix('candle_'),
                      zone_df.add_prefix('zone_')],
                     axis=1)


In [ ]:
candle_df.head()

In [ ]:
df_final.head(10)

In [ ]:
df_final.to_csv('direction_1h.csv',index= False)

In [ ]:
def flatten_label_row(labeled_row):
    flat_rows = []
    current_index = labeled_row['index']
    curr_price = labeled_row['candle detail']['close']
    touched_zone = labeled_row.get('zone_detail')
    past_zones = labeled_row.get('past_zones',[])
    if not past_zones:
        return []

    for zone in labeled_row['past_zones']:
        flat_row = {
            'candle_index': current_index,
            'zone_index': zone.get('index',None),
            'zone_type': zone.get('zone_type',None),
            'zone_mid_price': float(zone['mid_price']),
            'zone_high': float(zone['zone_high']),
            'zone_low': float(zone['zone_low']),
            'zone_age': current_index - zone['index'],
            'zone_range': float(zone['zone_high']) - float(zone['zone_low']),
            'zone_mid_distance': abs(curr_price - float(zone['mid_price'])),
            'price_above_zone': (curr_price > float(zone['zone_high'])),
            'price_inside_zone': (float(zone['zone_low']) < curr_price < float(zone['zone_high'])),
            'candle_close': curr_price,
            'candle_open': labeled_row['candle detail']['open'],
            'candle_high': labeled_row['candle detail']['high'],
            'candle_low': labeled_row['candle detail']['low'],
            'touched_index': labeled_row.get('touched_index'),
            'is_target': (
                touched_zone and
                zone['index'] == touched_zone['index'] and
                zone['zone_type'] == touched_zone['zone_type'] and
                abs(zone['zone_high'] - touched_zone['zone_high']) < 1e-6 and
                abs(zone['zone_low'] - touched_zone['zone_low']) < 1e-6
            )
        }

        flat_rows.append(flat_row)

    return flat_rows


In [ ]:
def flatten_all_rows(labeled_data):
    all_flat = []
    for row in labeled_data:
        all_flat.extend(flatten_label_row(row))
    return pd.DataFrame(all_flat)


In [ ]:
df_past_zones = flatten_all_rows(labels)

In [ ]:
df_past_zones = pd.read_csv('past_zone.csv')

In [ ]:
df_past_zones = df_past_zones.dropna()

In [ ]:
zone_type = list(df_past_zones['zone_type'].unique())
is_target = list(df_past_zones['is_target'].unique())
price_above_zone = list(df_past_zones['price_above_zone'].unique())
prive_inside_zone = list(df_past_zones['price_inside_zone'].unique())

In [ ]:
df_past_zones['zone_type'] = df_past_zones['zone_type'].apply(lambda x : zone_type.index(x))
df_past_zones['is_target'] = df_past_zones['is_target'].apply(lambda x : is_target.index(x))
df_past_zones['price_above_zone'] = df_past_zones['price_above_zone'].apply(lambda x : price_above_zone.index(x))
df_past_zones['price_inside_zone'] = df_past_zones['price_inside_zone'].apply(lambda x : prive_inside_zone.index(x))

In [ ]:
df_past_zones.columns

In [ ]:
df_past_zones.to_csv('past_zone.csv',index=False)

In [ ]:
df_swings = pd.DataFrame(swings)
df_swings.to_csv('swings.csv',index=False)

In [ ]:
df_final.notna().sum()

In [ ]:
labels[10]

In [ ]:
for i in labels:
    if i['index'] == 13670:
        print(i)

In [ ]:
fvg

In [ ]:
print(labels)

In [ ]:
df_with_labels = df_1h.join(labels, how='left')


In [ ]:
df.iloc[17254]

Data Combination of SMC Elements

In [ ]:
def smc_combine(df):
    merged = df.copy()
    merged = merged.reset_index(drop = True)
    merged['index'] = merged.index  # ensure index column exists
    fvg = detect_fvg(df)
    #fvg = check_fvg_reactions(df,fvg)
    ob = detect_order_blocks(df)
    #ob = check_ob_reactions(df,ob)
    swings = detect_swings(df,20)
    labeled_swings = label_structure_from_swings(swings)
    bos_choch=lookahead_bos_choch(labeled_swings,df,50)
    liq = detect_liquidity_zones(df,swings)
    #liq = check_liquidity_reactions(df,liq)

    fvg_df = pd.DataFrame(fvg).set_index('index')
    ob_df = pd.DataFrame(ob).set_index('index')
    bos_choch_df = pd.DataFrame(bos_choch).set_index('trigger_candle')
    labeled_swings_df = pd.DataFrame(labeled_swings).set_index('index')
    liq_df = pd.DataFrame(liq).set_index('start_index')
    merged = merged.merge(fvg_df, on='index', how='left', suffixes=('', '_fvg'))
    merged = merged.merge(ob_df, on='index', how='left', suffixes=('', '_ob'))
    merged = merged.merge(labeled_swings_df,on = 'index',how='left',suffixes=('','_swings'))
    merged = merged.merge(bos_choch_df, on='index', how='left', suffixes=('', '_bos_choch'))
    merged = merged.merge(liq_df, left_on='index', right_index=True, how='left', suffixes=('', '_liq'))
    merged['timestamp'] = df.index
    return merged

In [ ]:
smc_15m = smc_combine(df)
smc_1hr = smc_combine(df_1h)
smc_4hr = smc_combine(df_4h)
smc_1d = smc_combine(df_1d)

In [ ]:
def add_htf_features(base_df,htf_df,prefix):
    base_df = base_df.copy()
    htf_df = htf_df.copy()
    base_df = base_df.set_index('timestamp')
    htf_df = htf_df.set_index('timestamp')
    for feature in ['trend', 'fvg_high', 'fvg_low', 'ob_high', 'ob_low','level','swing_type']:
        if feature in htf_df.columns:
            base_df[f'{prefix}_{feature}'] = htf_df[feature].reindex(base_df.index, method='ffill')

    return base_df.reset_index()

In [ ]:
smc_15m = add_htf_features(smc_15m, smc_1hr, prefix='1h')
smc_15m = add_htf_features(smc_15m,smc_4hr, prefix='4h')
smc_15m = add_htf_features(smc_15m, smc_1d, prefix='1d')


In [ ]:
smc_15m.tail()

In [ ]:
smc_4hr.tail(20)

In [ ]:
smc_15m.to_csv('smc.csv',index=False)

In [ ]:
train_data = pd.read_csv('smc.csv')

# --- 1. Zone Confluence ---
train_data['fvg_ob_confluence'] = (
    (train_data['type'] == train_data['type_ob']) &
    train_data['type'].notna()
).astype(int)

train_data['fvg_liq_confluence'] = (
    (train_data['type'].notna()) &
    (train_data['type_liq'].notna()) &
    (abs(train_data['level'] - train_data['low']) < 1000)
).astype(int)

train_data['ob_liq_confluence'] = (
    (train_data['type_ob'].notna()) &
    (train_data['type_liq'].notna()) &
    (abs(train_data['level'] - train_data['low']) < 1000)
).astype(int)

# --- 2. Timing Features ---
train_data['time_since_bos_choch'] = train_data['index'] - train_data['trigger_candle']
train_data['delay_to_liquidity_sweep'] = train_data['swept_index'] - train_data['index']

# --- 3. Reaction Strengths ---
train_data['avg_max_price_move'] = train_data[[
    'max_price_move',
    'max_price_move_ob',
    'max_price_move_liq'
]].mean(axis=1)

train_data['has_strong_reaction'] = train_data['reaction_type'].isin(['clean bounce', 'bounce + break']).astype(int)

# --- 4. Labels ---
train_data['reaction_success'] = train_data['reaction_type'].isin(['clean bounce', 'bounce + break']).astype(int)
train_data['reaction_failed'] = (train_data['reaction_type'] == 'violation').astype(int)

train_data['setup_success'] = (
    (train_data['reaction_type_ob'].isin(['clean bounce'])) |
    (train_data['reaction_type'].isin(['clean bounce'])) |
    (train_data['reaction_type_liq'].isin(['clean bounce']))
).astype(int)

# Show the newly added features
train_data.tail()


In [ ]:
drop_cols = [
    'timestamp', 'index', 'price', 'trigger_candle', 'price_bos_choch',
    'trend_bos_choch', 'signal_bos_choch', 'trend_after_bos_choch','reaction_type',
    'reaction_type_ob',
    'reaction_type_liq','has_strong_reaction',
       'reaction_success', 'reaction_failed','price_retrace_ratio','max_price_move','price_retrace_ratio_ob',
       'max_price_move_ob','max_price_move_liq','avg_max_price_move','bounced','partial_mitigated','bounced_ob'
       ,'partial_mitigated_ob','bounced_liq' ,'violated','violated_ob','violated_liq'
]


In [ ]:
train_data.fillna({
    'structure_label': 'None',
    'signal': 'None',
    'trend': 'Unknown',

    'swing_type': 'None',
    'type': 'None',
    'type_ob': 'None',
    'type_liq': 'None'
}, inplace=True)

# Fill numeric NaNs
train_data.fillna(0, inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = [
    'structure_label', 'signal', 'trend',
    'swing_type', 'type', 'type_ob', 'type_liq'
]

le = LabelEncoder()
for col in categorical_cols:
    train_data[col] = le.fit_transform(train_data[col].astype(str))


In [ ]:
X = df_past_zones.drop(columns=['reaction_type','index','zone_high','zone_low'])
Y = df_past_zones['reaction_type']

In [ ]:
X = df_past_zones.drop(columns=['is_target','candle_index','zone_index','touched_index','zone_type'])
Y = df_past_zones['is_target']


In [ ]:
X = X.reset_index(drop= True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode all object columns
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# Cast entire DataFrame to float32
X = X.astype('float32')


In [ ]:
# Ensure all features are either int or float
cols = list(X.columns)
cols


In [ ]:
correlations = df_zone_reaction.corr()['reaction_type'].sort_values(ascending=False)
print(correlations)


In [ ]:
from sklearn.model_selection import train_test_split

train_x,val_x,train_y,val_y = train_test_split(X,Y,test_size=0.7)


In [ ]:
val_x.tail(5)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.05,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(train_x, train_y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=150,
    max_depth=6,
    random_state=42,
)

model.fit(train_x, train_y)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# After training
y_pred = model.predict(val_x)
y_proba = model.predict_proba(val_x)[:,1]

print(classification_report(val_y, y_pred))
print("AUC:", roc_auc_score(val_y, y_proba))


In [ ]:
# Get feature importance as dictionary
importance_dict = model.get_booster().get_score(importance_type='gain')  # You can also use: 'weight', 'cover', 'total_gain', 'total_cover'

# Convert to DataFrame for easier viewing
importance_df = pd.DataFrame.from_dict(importance_dict, orient='index', columns=['Importance'])
importance_df.index.name = 'Feature'
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

In [ ]:
reaction_type

In [ ]:
model.feature_importances_

In [ ]:
X.iloc[-40]

In [ ]:
model.predict(X.iloc[-41].values.reshape(1,-1))


In [ ]:
from joblib import dump, load


dump(model,'smcXGBoost.pkl')
print("Saved successfully")

In [ ]:
for k in train_data.iloc[50].keys():
    print(f'{k} : {train_data.iloc[50][k]}')